<a href="https://colab.research.google.com/github/RicardoMiles/Dissertation-template/blob/main/%E2%80%9Cchange_dropout_checkpoint_ipynb%E2%80%9Dcopy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/SummerProject/src')
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/SummerProject/src


In [9]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [10]:
# 定义包含CSV文件的文件夹路径
current_dir = os.getcwd()
root_dir = os.path.dirname(current_dir)
data_dir = os.path.join(root_dir, 'Data')
left_data = os.path.join(data_dir,'left')
right_data = os.path.join(data_dir,'right')

folder_path = left_data

# 获取left文件夹中所有CSV文件的列表
csv_files_left = [os.path.join(left_data, file) for file in os.listdir(left_data) if file.endswith('.csv')]
# 获取right文件夹中所有CSV文件的列表
csv_files_right = [os.path.join(right_data, file) for file in os.listdir(right_data) if file.endswith('.csv')]

# 初始化存储所有特征和标签的列表
all_features = []
all_labels = []

# 初始化CSV文件计数器
csv_files_count = 0

In [11]:
print(f"Number of left files: {len(csv_files_left)}")
print(f"Number of right files: {len(csv_files_right)}")

Number of left files: 26
Number of right files: 26


In [12]:
# 定义特征提取函数
def extract_features(group):
    features = []
    # 时间域特征
    for col in group.columns[2:]:
        data = group[col].values
        features.append(data.mean())
        features.append(data.std())
        features.append(stats.skew(data))
        features.append(stats.kurtosis(data))
    return features

In [13]:
# 初始化CSV文件计数器
csv_files_count = 0

# 处理left文件夹中的CSV文件
for file_name in csv_files_left:
    df = pd.read_csv(file_name)
    csv_files_count += 1  # 增加文件计数
    # 将数据按照epoch分组
    grouped = df.groupby(df.columns[1])  # 第二列是epoch分组的列
    for name, group in grouped:
        if len(group) == 101:
            # 提取特征
            features = group.iloc[:, 2:].values  # 第三列及以后是EEG通道信号
            all_features.append(features)
            # 获取标签
            all_labels.append('left')

# 处理right文件夹中的CSV文件
for file_name in csv_files_right:
    df_right = pd.read_csv(file_name)
    csv_files_count += 1  # 增加文件计数
    # 将数据按照epoch分组
    grouped_right = df_right.groupby(df_right.columns[1])  # 第二列是epoch分组的列
    for name, group in grouped_right:
        if len(group) == 101:
            # 提取特征
            features = group.iloc[:, 2:].values  # 第三列及以后是EEG通道信号
            all_features.append(features)
            # 获取标签
            all_labels.append('right')

# 将特征和标签转换为NumPy数组
features_array = np.array(all_features)
labels_array = np.array(all_labels)

# 打印处理的CSV文件数量
print(f"Total CSV files processed: {csv_files_count}")

Total CSV files processed: 52


In [14]:
# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(features_array, labels_array, test_size=0.2, random_state=42)

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# 初始化标签编码器
label_encoder = LabelEncoder()

# 将训练集和测试集的标签转换为数值
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# 确保转换后的标签是正确的数据类型
y_train_encoded = y_train_encoded.astype('int32')
y_test_encoded = y_test_encoded.astype('int32')

In [16]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, InputLayer
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# 假设 X_train, X_test, y_train, y_test 已经定义并准备好了
# 构建LSTM模型
model = Sequential()
model.add(InputLayer(input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=30, return_sequences=True))
model.add(Dropout(0.4))  # 添加Dropout层
model.add(LSTM(units=30))
model.add(Dropout(0.4))  # 添加Dropout层
model.add(Dense(1, activation='sigmoid'))  # 二分类问题

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 早停法
# early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

# 训练模型
model.fit(X_train, y_train_encoded, epochs=35, batch_size=64, validation_data=(X_test, y_test_encoded))

# 评估模型
loss, accuracy = model.evaluate(X_test, y_test_encoded, verbose=0)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/35
91/91 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.5019 - loss: 0.7121 - val_accuracy: 0.5208 - val_loss: 0.6923
Epoch 2/35
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5461 - loss: 0.6901 - val_accuracy: 0.5229 - val_loss: 0.6920
Epoch 3/35
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5596 - loss: 0.6853 - val_accuracy: 0.5187 - val_loss: 0.6935
Epoch 4/35
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5641 - loss: 0.6814 - val_accuracy: 0.5270 - val_loss: 0.6910
Epoch 5/35
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.5801 - loss: 0.6719 - val_accuracy: 0.5242 - val_loss: 0.6923
Epoch 6/35
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5963 - loss: 0.6627 - val_accuracy: 0.5339 - val_loss: 0.6999
Epoch 7/35
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6095 - loss: 0.6538 - val_accuracy: 0.5097 - val_loss: 0.7019
Epoch 8/35
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.6053 - loss: 0.6484 - val_accuracy: 0.5332 - v

In [17]:
# 预测测试集
predictions = model.predict(X_test)

# 将概率转换为类别
predicted_classes = (predictions > 0.5).astype("int32")

# 打印预测结果
print(predicted_classes)

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
[[0]
 [1]
 [0]
 ...
 [1]
 [1]
 [0]]


In [18]:
# 计算精确度、召回率和F1分数
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

precision = precision_score(y_test_encoded, predicted_classes)
recall = recall_score(y_test_encoded, predicted_classes)
f1 = f1_score(y_test_encoded, predicted_classes)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# 计算混淆矩阵
cm = confusion_matrix(y_test_encoded, predicted_classes)
print("Confusion Matrix:")
print(cm)

Precision: 0.4797768479776848
Recall: 0.4942528735632184
F1 Score: 0.4869072894550602
Confusion Matrix:
[[375 373]
 [352 344]]
